In [ ]:
import time
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def insertion_sort(arr):
    arr_copy = arr.copy()
    for i in range(1, len(arr_copy)):
        key = arr_copy[i]
        j = i - 1
        while j >= 0 and arr_copy[j] > key:
            arr_copy[j + 1] = arr_copy[j]
            j -= 1
        arr_copy[j + 1] = key
    return arr_copy

def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]
    left_sorted = merge_sort(left_half)
    right_sorted = merge_sort(right_half)
    return merge(left_sorted, right_sorted)

def merge(left, right):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result

def bubble_sort(arr):
    arr_copy = arr.copy()
    n = len(arr_copy)
    for i in range(n):
        for j in range(0, n - i - 1):
            if arr_copy[j] > arr_copy[j + 1]:
                arr_copy[j], arr_copy[j + 1] = arr_copy[j + 1], arr_copy[j]
    return arr_copy

def python_timsort(arr):
    return sorted(arr.copy())

In [ ]:
def generate_random_array(size, min_val=1, max_val=10000):
    return [random.randint(min_val, max_val) for _ in range(size)]

def measure_sorting_time(sort_function, array):
    start_time = time.time()
    result = sort_function(array)
    end_time = time.time()
    assert result == sorted(array), f"Sorting failed for {sort_function.__name__}"
    return end_time - start_time

def run_experiment(array_sizes, num_trials=3):
    results = []
    for size in array_sizes:
        print(f"Testing array size: {size}")
        insertion_times = []
        merge_times = []
        bubble_times = []
        timsort_times = []
        for trial in range(num_trials):
            test_array = generate_random_array(size)
            try:
                insertion_time = measure_sorting_time(insertion_sort, test_array)
                insertion_times.append(insertion_time)
            except Exception as e:
                print(f"Error in Insertion Sort trial {trial+1}: {e}")
                insertion_times.append(float('inf'))
            try:
                merge_time = measure_sorting_time(merge_sort, test_array)
                merge_times.append(merge_time)
            except Exception as e:
                print(f"Error in Merge Sort trial {trial+1}: {e}")
                merge_times.append(float('inf'))
            if size <= 2000:
                try:
                    bubble_time = measure_sorting_time(bubble_sort, test_array)
                    bubble_times.append(bubble_time)
                except Exception as e:
                    print(f"Error in Bubble Sort trial {trial+1}: {e}")
                    bubble_times.append(float('inf'))
            else:
                bubble_times.append(float('inf'))
            try:
                timsort_time = measure_sorting_time(python_timsort, test_array)
                timsort_times.append(timsort_time)
            except Exception as e:
                print(f"Error in Timsort trial {trial+1}: {e}")
                timsort_times.append(float('inf'))
        def safe_average(times):
            valid_times = [t for t in times if t != float('inf')]
            return sum(valid_times) / len(valid_times) if valid_times else float('inf')
        avg_insertion = safe_average(insertion_times)
        avg_merge = safe_average(merge_times)
        avg_bubble = safe_average(bubble_times)
        avg_timsort = safe_average(timsort_times)
        results.append({
            'Array Size': size,
            'Insertion Sort': avg_insertion,
            'Merge Sort': avg_merge,
            'Bubble Sort': avg_bubble,
            'Timsort': avg_timsort
        })
    return pd.DataFrame(results)

In [ ]:
def plot_results(df):
    plt.figure(figsize=(15, 10))
    plt.subplot(2, 2, 1)
    if any(df['Insertion Sort'] != float('inf')):
        plt.plot(df['Array Size'], df['Insertion Sort'], 'ro-', label='Insertion Sort (O(n²))', linewidth=2, markersize=6)
    if any(df['Merge Sort'] != float('inf')):
        plt.plot(df['Array Size'], df['Merge Sort'], 'bo-', label='Merge Sort (O(n log n))', linewidth=2, markersize=6)
    if any(df['Bubble Sort'] != float('inf')):
        plt.plot(df['Array Size'], df['Bubble Sort'], 'mo-', label='Bubble Sort (O(n²))', linewidth=2, markersize=6)
    if any(df['Timsort'] != float('inf')):
        plt.plot(df['Array Size'], df['Timsort'], 'go-', label='Timsort (O(n log n))', linewidth=2, markersize=6)
    plt.xlabel('Array Size')
    plt.ylabel('Time (seconds)')
    plt.title('Sorting Algorithm Performance Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.subplot(2, 2, 2)
    finite_sizes = df['Array Size']
    if any(df['Insertion Sort'] != float('inf')):
        finite_insertion = df[df['Insertion Sort'] != float('inf')]
        plt.loglog(finite_insertion['Array Size'], finite_insertion['Insertion Sort'], 'ro-', label='Insertion Sort', linewidth=2)
    if any(df['Merge Sort'] != float('inf')):
        finite_merge = df[df['Merge Sort'] != float('inf')]
        plt.loglog(finite_merge['Array Size'], finite_merge['Merge Sort'], 'bo-', label='Merge Sort', linewidth=2)
    if any(df['Bubble Sort'] != float('inf')):
        finite_bubble = df[df['Bubble Sort'] != float('inf')]
        plt.loglog(finite_bubble['Array Size'], finite_bubble['Bubble Sort'], 'mo-', label='Bubble Sort', linewidth=2)
    if any(df['Timsort'] != float('inf')):
        finite_timsort = df[df['Timsort'] != float('inf')]
        plt.loglog(finite_timsort['Array Size'], finite_timsort['Timsort'], 'go-', label='Timsort', linewidth=2)
    plt.xlabel('Array Size (log scale)')
    plt.ylabel('Time (seconds, log scale)')
    plt.title('Log-Log Scale: Theoretical Complexity Visualization')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.subplot(2, 2, 3)
    if all(df['Merge Sort'] != float('inf')) and all(df['Insertion Sort'] != float('inf')):
        ratios = df['Insertion Sort'] / df['Merge Sort']
        plt.plot(df['Array Size'], ratios, 'purple', marker='o', linewidth=2, label='Insertion/Merge Ratio')
        plt.xlabel('Array Size')
        plt.ylabel('Time Ratio')
        plt.title('Performance Ratio: O(n²) vs O(n log n)')
        plt.legend()
        plt.grid(True, alpha=0.3)
    plt.subplot(2, 2, 4)
    plt.axis('off')
    summary_data = []
    for idx, row in df.iterrows():
        if row['Insertion Sort'] != float('inf') and row['Merge Sort'] != float('inf'):
            speedup = row['Insertion Sort'] / row['Merge Sort']
            summary_data.append([f"{row['Array Size']:,}", f"{speedup:.1f}x"])
    if summary_data:
        table = plt.table(cellText=summary_data,
                         colLabels=['Array Size', 'Speedup (Merge vs Insertion)'],
                         loc='center',
                         cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        plt.title('Performance Speedup Summary')
    plt.tight_layout()
    plt.show()

In [ ]:
def print_detailed_analysis(df):
    print("\n" + "="*60)
    print("DETAILED ANALYSIS AND INSIGHTS")
    print("="*60)
    max_size = df['Array Size'].max()
    last_row = df[df['Array Size'] == max_size].iloc[0]
    print(f"\nLargest array tested: {max_size:,} elements")
    print(f"Number of different array sizes tested: {len(df)}")
    if last_row['Insertion Sort'] != float('inf') and last_row['Merge Sort'] != float('inf'):
        speedup = last_row['Insertion Sort'] / last_row['Merge Sort']
        print(f"\nPerformance on {max_size:,} elements:")
        print(f"  - Insertion Sort: {last_row['Insertion Sort']:.4f} seconds")
        print(f"  - Merge Sort: {last_row['Merge Sort']:.4f} seconds")
        print(f"  - Speedup factor: {speedup:.1f}x faster")
    print(f"\nTheoretical Time Complexities:")
    print(f"  - Insertion Sort: O(n²)")
    print(f"  - Bubble Sort: O(n²)")
    print(f"  - Merge Sort: O(n log n)")
    print(f"  - Timsort: O(n log n)")
    print(f"\nKey Observations:")
    print("1. O(n²) algorithms become impractical for large datasets")
    print("2. O(n log n) algorithms scale much better")
    print("3. Built-in Timsort is highly optimized for real-world data")
    print("4. For small arrays, simpler algorithms may be sufficient")
    if len(df) > 1:
        print(f"\nGrowth Rate Analysis:")
        for i in range(1, len(df)):
            size_ratio = df['Array Size'].iloc[i] / df['Array Size'].iloc[i-1]
            if (df['Insertion Sort'].iloc[i] != float('inf') and
                df['Insertion Sort'].iloc[i-1] != float('inf')):
                time_ratio_insertion = df['Insertion Sort'].iloc[i] / df['Insertion Sort'].iloc[i-1]
                print(f"  Insertion Sort: {size_ratio:.1f}x size → {time_ratio_insertion:.1f}x time")
            if (df['Merge Sort'].iloc[i] != float('inf') and
                df['Merge Sort'].iloc[i-1] != float('inf')):
                time_ratio_merge = df['Merge Sort'].iloc[i] / df['Merge Sort'].iloc[i-1]
                print(f"  Merge Sort:     {size_ratio:.1f}x size → {time_ratio_merge:.1f}x time")

In [ ]:
def main():
    print("="*60)
    print("SORTING ALGORITHM EFFICIENCY ANALYSIS")
    print("="*60)
    print("This program demonstrates the practical difference between")
    print("O(n²) and O(n log n) algorithms through empirical testing.")
    print("="*60)
    array_sizes = [100, 500, 1000, 2000, 5000, 10000]
    print(f"\nTesting array sizes: {array_sizes}")
    print(f"Number of trials per size: 3")
    print("\nGenerating test data and running experiments...")
    print("This may take a few moments for larger arrays...")
    results_df = run_experiment(array_sizes, num_trials=3)
    print("\n" + "="*60)
    print("EXPERIMENTAL RESULTS (Average Time in Seconds)")
    print("="*60)
    display_df = results_df.copy()
    for col in display_df.columns[1:]:
        display_df[col] = display_df[col].apply(
            lambda x: f"{x:.4f}" if x != float('inf') else "Too Slow"
        )
    print(display_df.to_string(index=False))
    print("\nGenerating visualizations...")
    plot_results(results_df)
    print_detailed_analysis(results_df)
    results_df.to_csv('sorting_algorithm_results.csv', index=False)
    print(f"\nResults saved to 'sorting_algorithm_results.csv'")
    print("\n" + "="*60)
    print("ANALYSIS COMPLETE!")
    print("="*60)
    print("Key takeaways:")
    print("✓ O(n²) algorithms grow quadratically with input size")
    print("✓ O(n log n) algorithms scale much better for large inputs")
    print("✓ Algorithm choice matters significantly for large datasets")
    print("✓ Built-in functions are often highly optimized")

if __name__ == "__main__":
    main()